In [9]:
import pickle
import os
import re, string, unicodedata
import nltk
!pip install contractions
!pip install inflect
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def DoctoDict():
  file_Names = os.listdir("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1")
  # print(file_lists)
  file_Paths = []
  for i in range(len(file_Names)):
    file_Paths.append("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1/"+file_Names[i])
  corpus = {}
  for i in range(len(file_Paths)):
    with open(file_Paths[i],encoding = "latin-1") as f_input:
      corpus[file_Names[i]] = [f_input.read()]
  return corpus

In [4]:
CORPUS = DoctoDict()
with open('CORPUS.pickle', 'wb') as handle:
    pickle.dump(CORPUS, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('/content/CORPUS.pickle', 'rb') as handle:
    # corpus = pickle.load(handle)

In [5]:
with open('/content/CORPUS.pickle', 'rb') as handle:
    corpus = pickle.load(handle)
len(corpus.keys())

468

In [7]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    text=re.sub('\n',' ',text)
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
    return tag_dict.get(tag)

def lemmatize(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        posTag = get_wordnet_pos(word)
        if posTag:
          lemma = lemmatizer.lemmatize(word,posTag)
        else:
          lemma = word
        lemmas.append(lemma)
    return lemmas


def preProcess_html(fileName):
    sample = denoise_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize(words)
    return words  

def clean_text(text):
    # text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z0-9A-Z]',' ',text)
    text=re.sub(' +',' ',text)
    return text

def preProcessotherfiles(fileName):
    sample = clean_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize(words)
    return words 


In [10]:
for i in corpus.keys():
  if i.endswith(".html") or i.endswith(".header"):
    corpus[i][0] = preProcess_html(corpus[i][0])
  else:
    corpus[i][0] = preProcessotherfiles(corpus[i][0])

In [11]:
len(corpus.keys())
# corpus.keys()

468

In [12]:
corpus['cmoutmou.txt'][0]

['country',
 'mouse',
 'town',
 'mouse',
 'upon',
 'time',
 'town',
 'mouse',
 'trip',
 'country',
 'met',
 'country',
 'mouse',
 'spent',
 'day',
 'together',
 'become',
 'friend',
 'country',
 'mouse',
 'take',
 'new',
 'friend',
 'meadow',
 'vegetable',
 'garden',
 'make',
 'sample',
 'good',
 'thing',
 'land',
 'never',
 'see',
 'beauty',
 'countryside',
 'town',
 'mouse',
 'thrill',
 'though',
 'country',
 'mouse',
 'plain',
 'food',
 'nearly',
 'fine',
 'usual',
 'meal',
 'thank',
 'friend',
 'lovely',
 'out',
 'invite',
 'country',
 'mouse',
 'visit',
 'town',
 'country',
 'mouse',
 'saw',
 'pantry',
 'friend',
 'house',
 'full',
 'ham',
 'cheese',
 'oil',
 'flour',
 'honey',
 'jam',
 'stack',
 'goody',
 'stood',
 'speechless',
 'surprise',
 'never',
 'see',
 'anything',
 'like',
 'wonderful',
 'thing',
 'eat',
 'course',
 'come',
 'ply',
 'guest',
 'tuck',
 'begin',
 'feast',
 'country',
 'mouse',
 'try',
 'stuff',
 'want',
 'taste',
 'everything',
 'find',
 'tummy',
 'full',
 

In [ ]:
corpus['cmoutmou.txt'][0]

['country',
 'mouse',
 'town',
 'mouse',
 'upon',
 'time',
 'town',
 'mouse',
 'trip',
 'country',
 'meet',
 'country',
 'mouse',
 'spend',
 'day',
 'together',
 'become',
 'friends',
 'country',
 'mouse',
 'take',
 'new',
 'friend',
 'meadows',
 'vegetable',
 'garden',
 'make',
 'sample',
 'good',
 'things',
 'land',
 'never',
 'see',
 'beauties',
 'countryside',
 'town',
 'mouse',
 'thrill',
 'though',
 'country',
 'mouse',
 'plain',
 'food',
 'nearly',
 'fine',
 'usual',
 'meals',
 'thank',
 'friend',
 'lovely',
 'out',
 'invite',
 'country',
 'mouse',
 'visit',
 'town',
 'country',
 'mouse',
 'saw',
 'pantry',
 'friend',
 'house',
 'full',
 'ham',
 'cheese',
 'oil',
 'flour',
 'honey',
 'jam',
 'stack',
 'goodies',
 'stand',
 'speechless',
 'surprise',
 'never',
 'see',
 'anything',
 'like',
 'wonderful',
 'things',
 'eat',
 'course',
 'come',
 'ply',
 'guest',
 'tuck',
 'begin',
 'feast',
 'country',
 'mouse',
 'try',
 'stuff',
 'want',
 'taste',
 'everything',
 'find',
 'tummy',


In [13]:
with open('DocTerms.pickle', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('/content/DocTerms.pickle', 'rb') as handle:
  #  docT = pickle.load(handle)

In [ ]:
docT['.windex.html'][0]

['e',
 'x',
 'f',
 'l',
 'e',
 'stories',
 'tristan',
 'farnon',
 'filename',
 'size',
 'description',
 'textfile',
 'request',
 'tristan',
 'farnon',
 'directory',
 'remove',
 'sadden',
 'end',
 'stories',
 'truly',
 'wonderful',
 'would',
 'post',
 'dark',
 'side',
 'moon',
 'ae',
 'always',
 'interest',
 'file',
 'post',
 'tristan',
 'always',
 'cause',
 'rush',
 'excitement',
 'place',
 'could',
 'tell',
 'would',
 'post',
 'one',
 'system',
 'would',
 'really',
 'busy',
 'tristan',
 'stories',
 'fill',
 'irony',
 'interest',
 'situations',
 'wellwritten',
 'dialogue',
 'pleasant',
 'change',
 'else',
 'would',
 'find',
 'interest',
 'cause',
 'destruction',
 'insult',
 'others',
 'entertain',
 'tristan',
 'farnon',
 'go',
 'excellent',
 'webbased',
 'work',
 'site',
 'leisuretown',
 'continue',
 'display',
 'excellent',
 'sense',
 'humor',
 'irony',
 'characterization',
 'flatout',
 'quality',
 'give',
 'highest',
 'recommendation']